In [43]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [44]:
house_dict = []

for page in range(17, 21):
    linked = 'https://www.zillow.com/ca/townhomes/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-153.847663625,"east":-84.765632375,"south":12.777434491046995,"north":56.013075839104054},"mapZoom":4,"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"filterState":{"isPreMarketPreForeclosure":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"isSingleFamily":{"value":false},"isManufactured":{"value":false},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isLotLand":{"value":false},"doz":{"value":"90"},"isApartment":{"value":false},"isPreMarketForeclosure":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-18-0__aiai24-0 einFCw")[1].text
            except:
                zestimate = 'null'
#             try:
#                 rent_zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[1].text
#             except:
#                 rent_zestimate = 'null'
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
#                 "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(3)
            
        except AttributeError as e:
            continue

print(house_dict)

https://www.zillow.com/homedetails/3800-Stocker-St-APT-32-View-Park-CA-90008/20566918_zpid/
zestimate $619,385
https://www.zillow.com/homedetails/20427-Paseo-Azul-Northridge-CA-91326/95585946_zpid/
zestimate $642,407
https://www.zillow.com/community/violet-at-canvas/2080511505_zpid/
zestimate null
https://www.zillow.com/homedetails/2243-Stonewood-Ct-San-Pedro-CA-90732/21301984_zpid/
zestimate $580,005
https://www.zillow.com/homedetails/764-Via-Colinas-Westlake-Village-CA-91362/16488489_zpid/
zestimate $639,921
https://www.zillow.com/community/uptown-in-livermore/2077104727_zpid/
zestimate null
https://www.zillow.com/homedetails/12-Portside-Irvine-CA-92614/25752585_zpid/
zestimate $889,001
https://www.zillow.com/homedetails/10509-Caminito-Basswood-San-Diego-CA-92131/16866939_zpid/
zestimate $629,999
https://www.zillow.com/community/solstice-at-day-creek-square/2085199166_zpid/
zestimate null
https://www.zillow.com/homedetails/6476-Summit-Village-Ln-UNIT-2-Simi-Valley-CA-93063/131677527_

zestimate null
https://www.zillow.com/community/rancho-milagro-at-pacific-highlands-ranch/2077486549_zpid/
zestimate null
https://www.zillow.com/homedetails/621-W-Foothill-Blvd-UNIT-49-Glendora-CA-91741/244502106_zpid/
zestimate $620,005
https://www.zillow.com/homedetails/219-Altura-Vis-Los-Gatos-CA-95032/19666516_zpid/
zestimate $1,489,647
https://www.zillow.com/homedetails/1742-Glendon-Ave-APT-C-Los-Angeles-CA-90024/20505148_zpid/
zestimate $1,195,008
https://www.zillow.com/homedetails/3001-S-Sycamore-St-APT-F-Santa-Ana-CA-92707/25684362_zpid/
zestimate $469,278
https://www.zillow.com/homedetails/2628-Kansas-Ave-APT-13-Santa-Monica-CA-90404/20473532_zpid/
zestimate $759,400
https://www.zillow.com/homedetails/13880-Sayre-St-UNIT-9-Los-Angeles-CA-91342/20072980_zpid/
zestimate $392,668
https://www.zillow.com/community/cerro-at-rancho-soleo/2080511488_zpid/
zestimate null
https://www.zillow.com/homedetails/1908-Plant-Ave-A-Redondo-Beach-CA-90278/20403336_zpid/
zestimate $1,216,191
https

zestimate $729,006
https://www.zillow.com/community/suwerte/2087663802_zpid/
zestimate null
https://www.zillow.com/homedetails/1360-Josselyn-Canyon-Rd-APT-33-Monterey-CA-93940/19358798_zpid/
zestimate $639,003
https://www.zillow.com/homedetails/1758-Pedrozzi-Cir-14-Livermore-CA-94551/2076826263_zpid/
zestimate $878,628
[{'Listing_type': 'Townhouse for sale', 'Price': '$610,000', 'Full_address': '3800 Stocker St APT 32, View Park, CA 90008', 'Street': '3800 Stocker St APT 32', 'City': 'ViewPark', 'State': 'CA', 'Zip_code': '90008', 'Bed': '2', 'Bath': '3', 'Square_Feet': '1,300', 'Lot': '2.52 Acres', 'Lot_conversion_to_sqft': 109771.2, 'House_type': 'Townhouse', 'Year_build': '1984', 'Parking': 'Carport, Covered', 'HOA': '$456/mo', 'Zestimate': '$619,385', 'Zillow_price/sqft': '$469', 'House_website': 'https://www.zillow.com/homedetails/3800-Stocker-St-APT-32-View-Park-CA-90008/20566918_zpid/'}, {'Listing_type': 'Townhouse for sale', 'Price': '$639,000', 'Full_address': '20427 Paseo Azu

In [45]:
house_data = pd.DataFrame(house_dict)
house_data

Listing_type      Price  \
0    Townhouse for sale   $610,000   
1    Townhouse for sale   $639,000   
2      New construction  $676,710+   
3            Contingent   $580,000   
4    Townhouse for sale   $639,918   
..                  ...        ...   
155    New construction  $432,435+   
156  Townhouse for sale   $729,000   
157    New construction  $499,900+   
158          Contingent   $639,000   
159  Townhouse for sale   $848,975   

                                          Full_address  \
0          3800 Stocker St APT 32, View Park, CA 90008   
1               20427 Paseo Azul, Northridge, CA 91326   
2                   Residence 3 Plan, Violet at Canvas   
3               2243 Stonewood Ct, San Pedro, CA 90732   
4          764 Via Colinas, Westlake Village, CA 91362   
..                                                 ...   
155                                Plan 2 Plan, Solmar   
156             8 Arabian Way, Scotts Valley, CA 95066   
157                               Plan 5 Plan, Suwerte   
158  1360 Josselyn Canyon Rd APT 33, Monterey, CA 9...   
159         1758 Pedrozzi Cir #14, Livermore, CA 94551   

                             Street             City State Zip_code Bed Bath  \
0            3800 Stocker St APT 32         ViewPark    CA    90008   2    3   
1                  20427 Paseo Azul       Northridge    CA    91326   2    3   
2                  Residence 3 Plan   VioletatCanvas  null     null   4    4   
3                 2243 Stonewood Ct         SanPedro    CA    90732   2    3   
4                   764 Via Colinas  WestlakeVillage    CA    91362   3    3   
..                              ...              ...   ...      ...  ..  ...   
155                     Plan 2 Plan           Solmar  null     null   2    3   
156                   8 Arabian Way     ScottsValley    CA    95066   3    2   
157                     Plan 5 Plan          Suwerte  null     null   3    4   
158  1360 Josselyn Canyon Rd APT 33         Monterey    CA    93940   2    3   
159           1758 Pedrozzi Cir #14        Livermore    CA    94551   4    4   

    Square_Feet          Lot Lot_conversion_to_sqft House_type Year_build  \
0         1,300   2.52 Acres                 109771  Townhouse       1984   
1         1,381   0.73 Acres                31798.8  Townhouse       2009   
2         2,179      No Data                   null  Townhouse       2021   
3         1,429  10.70 Acres                 466092  Townhouse       1979   
4         1,435   1,433 sqft                   1433  Townhouse       1988   
..          ...          ...                    ...        ...        ...   
155       1,310      No Data                   null  Townhouse       2021   
156       1,393   1,568 sqft                   1568  Townhouse       1985   
157       2,025      No Data                   null  Townhouse       2021   
158       1,778   1,454 sqft                   1454  Townhouse       1975   
159       1,929   2,500 sqft                   2500  Townhouse       2020   

                                Parking      HOA Zestimate Zillow_price/sqft  \
0                      Carport, Covered  $456/mo  $619,385              $469   
1            Garage - Attached, Covered  $382/mo  $642,407              $463   
2                               Covered     null      null              $311   
3            Garage - Attached, Covered  $463/mo  $580,005              $406   
4    Garage, Garage - Attached, Covered  $307/mo  $639,921              $446   
..                                  ...      ...       ...               ...   
155                             Covered  $305/mo      null              $330   
156           Garage, Garage - Attached  $391/mo  $729,006              $523   
157                             Covered  $266/mo      null              $247   
158                             Carport  $537/mo  $639,003              $359   
159                   Garage - Attached  $370/mo  $878,628              $440   

          

In [46]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q4-2020_Townhouse_Data_page17-20.csv", index=False, header=True)